<a href="https://colab.research.google.com/github/thanachai-npr/CE888/blob/main/lab4/my_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC

np.set_printoptions(precision=3)

In [3]:
jokes_df = pd.read_csv("https://raw.githubusercontent.com/albanda/CE888/master/lab4-recommender/jester-data-1.csv",  header = None)
df = jokes_df.replace(99, np.NaN)
df.drop([0], axis=1, inplace = True)
print(df)


        1     2     3     4     5     6    ...   95    96    97    98    99    100
0     -7.82  8.79 -9.66 -8.16 -7.52 -8.50  ...   NaN   NaN -5.63   NaN   NaN   NaN
1      4.08 -0.29  6.36  4.37 -2.38 -9.66  ... -0.19 -2.14  3.06  0.34 -4.32  1.07
2       NaN   NaN   NaN   NaN  9.03  9.27  ...   NaN   NaN   NaN   NaN   NaN   NaN
3       NaN  8.35   NaN   NaN  1.80  8.16  ...   NaN   NaN   NaN   NaN   NaN   NaN
4      8.50  4.61 -4.17 -5.39  1.36  1.60  ...  5.73  1.55  3.11  6.55  1.80  1.60
...     ...   ...   ...   ...   ...   ...  ...   ...   ...   ...   ...   ...   ...
24978  0.44  7.43  9.08  2.33  3.20  6.75  ...  8.45  9.03  6.55  8.69  8.79  7.43
24979  9.13 -8.16  8.59  9.08  0.87 -8.93  ...  9.22 -8.20 -7.23 -8.59  9.13  8.45
24980   NaN   NaN   NaN   NaN -7.77   NaN  ...   NaN   NaN   NaN   NaN   NaN   NaN
24981   NaN   NaN   NaN   NaN -9.71   NaN  ...   NaN   NaN   NaN   NaN   NaN   NaN
24982  2.43  2.67 -3.98  4.27 -2.28  7.33  ...   NaN   NaN   NaN   NaN   NaN   NaN

[24

In [4]:
arr = df.values
print(arr)

[[-7.82  8.79 -9.66 ...   nan   nan   nan]
 [ 4.08 -0.29  6.36 ...  0.34 -4.32  1.07]
 [  nan   nan   nan ...   nan   nan   nan]
 ...
 [  nan   nan   nan ...   nan   nan   nan]
 [  nan   nan   nan ...   nan   nan   nan]
 [ 2.43  2.67 -3.98 ...   nan   nan   nan]]


In [5]:
def replace(orig, percentage=0.1):
  """
  Replaces 'percentage'% of the original values in 'orig' with 99's
  :param orig: original data array
  :param percentage: percentage of values to replace (0<percentage<1)
  """
  new_data = orig.copy()
  rated = np.where(~np.isnan(orig))
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  for i in idx:
    assert new_data[rated[0][i], rated[1][i]] != np.nan
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])

In [6]:
new_arr, idx = replace(arr, 0.1)

In [7]:
print(new_arr)

[[99.    8.79 -9.66 ...   nan   nan   nan]
 [ 4.08 -0.29  6.36 ...  0.34 -4.32 99.  ]
 [  nan   nan   nan ...   nan   nan   nan]
 ...
 [  nan   nan   nan ...   nan   nan   nan]
 [  nan   nan   nan ...   nan   nan   nan]
 [99.    2.67 -3.98 ...   nan   nan   nan]]


In [8]:
print(arr)

[[-7.82  8.79 -9.66 ...   nan   nan   nan]
 [ 4.08 -0.29  6.36 ...  0.34 -4.32  1.07]
 [  nan   nan   nan ...   nan   nan   nan]
 ...
 [  nan   nan   nan ...   nan   nan   nan]
 [  nan   nan   nan ...   nan   nan   nan]
 [ 2.43  2.67 -3.98 ...   nan   nan   nan]]


In [9]:
n_latent_factors = 2

user_ratings = new_arr
# Initialise as random values
latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))

In [10]:
latent_item_features

array([[2.084e-01, 7.275e-01],
       [2.308e-01, 2.317e-01],
       [1.644e-01, 4.462e-01],
       [9.254e-01, 1.828e-01],
       [5.647e-01, 8.834e-01],
       [5.248e-01, 6.909e-01],
       [9.214e-02, 9.018e-01],
       [4.314e-01, 9.354e-01],
       [9.336e-01, 9.772e-01],
       [7.053e-01, 2.671e-01],
       [7.234e-01, 2.848e-01],
       [4.931e-01, 2.554e-01],
       [7.525e-02, 3.144e-01],
       [4.870e-01, 3.324e-01],
       [3.286e-01, 7.627e-01],
       [5.861e-01, 5.916e-01],
       [7.337e-01, 7.681e-01],
       [9.473e-01, 4.866e-01],
       [1.560e-01, 8.368e-01],
       [8.204e-03, 1.746e-01],
       [2.823e-01, 9.882e-01],
       [8.802e-01, 2.479e-01],
       [9.705e-01, 6.087e-01],
       [3.514e-01, 8.568e-01],
       [6.501e-01, 5.883e-01],
       [4.825e-01, 7.328e-01],
       [4.950e-01, 6.266e-01],
       [3.143e-01, 6.964e-01],
       [8.783e-01, 3.226e-01],
       [6.369e-01, 8.073e-02],
       [8.275e-01, 5.830e-01],
       [2.678e-01, 7.758e-01],
       [

In [11]:
latent_user_preferences

array([[0.85 , 0.357],
       [0.202, 0.852],
       [0.442, 0.774],
       ...,
       [0.551, 0.312],
       [0.529, 0.086],
       [0.418, 0.28 ]])

In [12]:
def predict_rating(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)


def train(user_id, item_id, rating, alpha=0.0001):
    #print(item_id)
    predicted_rating = predict_rating(user_id, item_id)
    err =  predicted_rating - rating
    #print(err)
    user_pref_values = latent_user_preferences[user_id]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err

def sgd(iterations):
    """ 
    Iterate over all users and all items and train for 
    a certain number of iterations
    """
    
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
              rating = user_ratings[user_id, item_id]
              if not np.isnan(rating) and rating != 99:
                err = train(user_id, item_id, rating)
                error.append(err)
    mse = (np.array(error) ** 2).mean()          
    print(mse)

In [13]:
sgd(5)

24.519446003293705


In [14]:
df.iloc[18473, 0]

nan

In [15]:
item_features_df = pd.read_excel('movies_latent_factors.xlsx', sheet_name='Items',index_col='Movie ID')
user_ratings_df = pd.read_excel('movies_latent_factors.xlsx', sheet_name='Users',index_col='User')
item_features_df = item_features_df.drop('Title', axis=1)

In [16]:
item_features_df = item_features_df.values
print(item_features_df)


[[-1.522 -1.039  2.027 ...  0.404  1.13  -0.249]
 [-0.342 -0.297 -0.386 ... -0.564  0.507  1.275]
 [-2.241 -0.439 -2.275 ... -0.33  -2.209 -0.419]
 ...
 [ 0.812 -0.893  0.669 ...  0.642 -1.356  0.068]
 [ 1.161 -0.716  0.713 ...  0.291 -1.217 -0.288]
 [ 1.734  0.265 -0.003 ...  0.334 -0.223 -0.023]]


In [17]:
user_ratings_df = user_ratings_df.values
print(user_ratings_df)

[[-2.040e-01  1.611e-01 -9.045e-02  1.385e-01 -1.629e-01  1.639e-01
   5.150e-02 -8.858e-02  1.268e-01  6.597e-02  8.501e-02  3.554e-01
   7.108e-03 -1.187e-01 -3.913e-02]
 [-1.897e-01 -1.790e-01 -9.149e-02 -8.228e-04 -3.265e-02  1.772e-01
  -9.812e-02 -6.828e-02 -1.157e-02  1.209e-01 -9.931e-03 -4.861e-02
   4.592e-02  1.137e-01  1.799e-01]
 [-1.153e-01 -9.089e-02 -5.313e-02  1.847e-02 -6.808e-02 -4.828e-03
   1.130e-01  1.021e-01  3.476e-02  6.926e-04 -7.371e-02 -1.946e-02
   1.084e-01  5.447e-02 -1.096e-01]
 [-2.275e-01 -2.725e-01 -1.723e-02  5.432e-02  2.148e-01 -7.264e-02
  -3.312e-02 -8.651e-02 -1.315e-01  1.804e-01  9.589e-02 -8.240e-02
   3.677e-02 -1.654e-01  5.069e-02]
 [-1.462e-02 -1.022e-01 -1.079e-01  1.558e-01 -1.234e-01 -1.182e-01
  -1.355e-02 -5.062e-02  5.870e-02 -1.596e-01 -1.424e-01 -1.328e-01
  -3.990e-02  1.291e-01  1.027e-01]
 [-2.030e-02 -3.192e-02 -3.696e-02  3.369e-02  1.745e-04 -3.178e-03
   5.447e-02  4.542e-02 -5.331e-02  3.437e-03 -4.175e-02  5.973e-02
  -2

In [18]:
##Q7
rating = user_ratings_df[15].dot(item_features_df[46])
print(rating)

-0.09541436887627988


In [19]:
##Q8
rating = user_ratings_df[22].dot(item_features_df[54])
print(rating)

0.3682088030139018


In [20]:
##Q9
rating = []
for i in range(len(item_features_df)):
  rating.append(user_ratings_df[14].dot(item_features_df[i]))

In [21]:
##Q10

2nd TRY


In [22]:
##Q4
df.iloc[12435, 16]

-6.41

In [32]:
##Q5
item_features_df[46][0]

-0.34266902985219055

In [39]:
##Q6
np.argsort(item_features_df[:,2])

array([47,  2, 39, 10, 33, 13, 26, 69, 89, 54, 38, 60, 84, 83, 91, 27, 81,
       63, 40, 49, 76, 73, 71, 22, 45, 75, 87, 31, 72, 35, 94, 44,  1, 48,
       21, 68, 96, 53, 92, 56,  8, 93, 64, 23, 90,  3, 74, 50, 70, 99, 51,
       41, 85, 65, 80, 11, 67,  6, 46, 66, 62, 55, 25, 95, 61, 32,  4, 20,
       52, 30, 12, 34, 28, 42, 82,  9, 37, 88, 43, 18, 17, 97, 59, 98,  7,
       19, 29, 24, 79, 86, 58, 57, 36,  0,  5, 78, 16, 77, 15, 14])

In [40]:
##Q7
rating = user_ratings_df[20].dot(item_features_df[2])
print(rating)

0.23490491668438457


In [41]:
##Q8
rating = user_ratings_df[14].dot(item_features_df[8])
print(rating)

-0.00686305800686321


In [42]:
##Q9
rating = []
for i in range(len(item_features_df)):
  rating.append(user_ratings_df[9].dot(item_features_df[i]))
print(np.argsort(rating)[::-1])


[51 50 47 21 41  8  1 30  3 25 27 70 91 20 22 77  6  4 46 95 19  0 78 59
 39 83 84 71 61 72 53 23 57 67 58 55 96 79 35 73  7 49 32 31 65 66 54 18
 52 33 38 64 34 24 17 48 10 80 60 12  5 16 36 14 88 69 98 97 28 13 15 42
 11 62 29 93 43  9 86 44 89  2 40 37 82 45 87 76 56 92 74 81 68 85 75 99
 26 63 94 90]


In [43]:
##Q10
rating = []
for i in range(len(item_features_df)):
  rating.append(user_ratings_df[0].dot(item_features_df[i]))
print(np.argsort(rating)[::-1])

[ 9 11 33  2 65 59 53 27 38 87 83 64 31 90 70 12 32 69 77 34 44 41  0 73
 60 84 48 75 46 78 54 26  8 17 21 95 35 74 28 94 10  1  7 63 85 92 99 45
 39 89 18 80 25 66 62 55 57 58  6 30 88 40 68 91 82 22 67 76 93 20 24 52
  3 50 96 81 29 23 71 14  5 97 61 16 19 36 56 15 13 98  4 49 42 72 43 37
 86 79 47 51]
